<a href="https://colab.research.google.com/github/massinoLight/tp_AlgoBD/blob/main/Tp_Arbre__de_d%C3%A9cision_pyspark_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 62.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a1abca206a7471580e1e4a06fa8efe9419b5dd059329962a3c08d75c21ebf25d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import requests 
file_url = "https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_libsvm_data.txt"
	
r = requests.get(file_url, stream = True) 

with open("/content/sample_data/sample_libsvm_data.txt", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

In [3]:
from pyspark import SparkContext
# $example on$
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

sc = SparkContext(appName="PythonDecisionTreeClassification")

##chargé le jeu de données et diviser en données d'entrainements et de tests 

In [9]:

data = MLUtils.loadLibSVMFile(sc, '/content/sample_data/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

    

# Save and load model



##derterminer notre modéle (nombre de class=2)

In [ ]:
# Train a DecisionTree model.
    #  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                         impurity='gini', maxDepth=5, maxBins=32)

    # Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
        lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())

##pyspark permet de sauvegarder le modéle entrainé sauvegarder le modéle et de le charger en mémoire pour une autre utilisation

In [10]:
model.save(sc, "model/DT/myDecisionTreeClassificationModel")
sameModel = DecisionTreeModel.load(sc, "model/DT/myDecisionTreeClassificationModel")